<h1>Проект по курсу "Анализ данных в бизнесе"</h1>

<h2>Тема: <b>"Автоматизация работы с документами: извлечение сущностей и фактов из сообщений о раскрытии"</b></h1>

<h4>Выполнили: Митченко Анастасия, Неминская Софья, Уали Акжан</h4>
<h4 >Руководитель: Дудников Константин</h4>

####Извлечение решения из страниц html

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import glob
import io
import re

In [ ]:
data = []
files = []
for filename in glob.glob("htmls/*.html"):
    with io.open(filename, encoding='utf-8') as htmlfile:
        html = htmlfile.read()
        soup = BeautifulSoup(html, 'html.parser')
        files += [soup]
        content = soup.find("div", {"id": "cont_wrap"})
        if content:
            content = content.text.rstrip()
        content = re.sub(r'\n', ' ', content)
        content = re.sub(r'\t', ' ', content)
        content = re.sub(r'\xa0', ' ', content)
        data.append(" ".join(content.split()))

####Полное наименование эмитента

In [ ]:
full_names = []
for html in data:
    full_name = re.findall(r'(?<=Полное фирменное наименование эмитента \(для некоммерческой организации [–—-] наименование\): )(.+?) 1\.2\.', html)
    if len(full_name) == 0:
        full_name = re.findall(r'(?<=Полное фирменное наименование эмитента: )(.+?) 1\.2\.', html)
        if len(full_name) == 0:
            full_name = re.findall(r'(?<=Полное фирменное наименование эмитента \(для некоммерческой организации наименование\): )(.+?) 1\.2\.', html)
            if len(full_name) == 0:
                full_name = re.findall(r'(?<=Полное фирменное наименование эмитента \(для некоммерческой организации [—–] наименование\) )(.+?) 1\.2\.', html)
                if len(full_name) == 0:
                    full_name = re.findall(r'(?<=Полное фирменное наименование эмитента )(.+?) 1\.2\.', html)
                    if len(full_name) == 0:
                        full_names.append(False)
                        continue
    full_names.append(full_name[0])

####Сокращенное наименование эмитента

In [ ]:
names = []
for html in data:
   name = re.findall('(?<=Сокращенное фирменное наименование эмитента )\w+\s\“\w+?\s?\“\w+?\b\”|(?<=Сокращенное фирменное наименование эмитента )\w+\s\"\w+\"|(?<=Сокращенное фирменное наименование эмитента )\w+\s«?\w+?\s?«?\w+?\s?\w+?\»|(?<=Сокращенное фирменное наименование эмитента )\w+\s«?\w+?\s?\w+?\s?«?\w+?\S?\s?\w+?\»|(?<=Сокращенное фирменное наименование эмитента )\w+\s\S?\w+\S?|(?<=Сокращенное фирменное наименование эмитента: )\w+\s?\"?\s?\w+?\s?\w+?\s?\S?\s?\"?\w+?\S?\s?\w+?\S?\s?\"?\s?\w+?\s?\w+?\"|(?<=Сокращенное фирменное наименование эмитента..)\w+\s«?\w+?\S?\s?\w+?\s?«?\S?\w*?\s?\»?\w+?»|(?<=Сокращенное фирменное наименование эмитента )\w+\s?\w+?\s?\"?\w+?\s?\w+?\S?\s?\w+?\S?\s?\w+?\S?\w+?\"|(?<=Сокращенное фирменное наименование эмитента )\w+\s«?\w+?\s?\w+?\S?\s?\w+?\S?\s?\w+?\»|(?<=Сокращенное фирменное наименование эмитента: )\w+\s\"\w+?\s?\S?\s?\w*?\S?\w+?\"|(?<=Сокращенное фирменное наименование эмитента: )\w+\s\w+\s?\w+?\b|(?<=Сокращенное фирменное наименование эмитента )\w+\s?\“\w+?\S?\s?\“?\w+?\s?\w+?\”|(?<=Сокращенное фирменное наименование эмитента )\w+\s?\"?\w+?\"?|(?<=Сокращенное фирменное наименование эмитента: )\w+\s\S?\w+\S?|(?<=Сокращенное фирменное наименование эмитента: )\"\w+?\s?\S?\s?\w*?\"\s.\w+.', html)
   if len(name) > 0:
       names.append(name[0])
   else:
       names.append(False)

####Адрес эмитента

In [ ]:
addresses = []
for html in data:
    address = re.findall(r'(?<=Место нахождения эмитента: )(.+?) 1\.4\.', html)
    if len(address) == 0:
        address = re.findall(r'(?<=Место нахождения эмитента )(.+?) 1\.4\.', html)
        if len(address) == 0:
            address = re.findall(r'(?<=Место нахождения: )(.+?) 1\.4\.', html)
            if len(address) == 0:
                addresses.append(False)
                continue
    addresses.append(address[0])

####ИНН эмитента

In [ ]:
innPattern = re.compile(r'(?<=инн эмитента: )\d{10}\b|(?<=инн эмитента )\d{10}\b|(?<=инн эмитента [-\–] )\d{10}\b')

In [ ]:
inns = []
for html in data:
    inn = re.search(innPattern, html.lower())
    if inn:
        inns.append(inn[0])
    else:
        inns.append(False)

####ОГРН эмитента

In [ ]:
ogrnPattern = re.compile(r'(?<=огрн эмитента: )\d{13}\b|(?<=огрн эмитента )\d{13}\b|(?<=огрн эмитента [-\–] )\d{13}\b')

In [ ]:
ogrns = []
for html in data:
    ogrn = re.search(ogrnPattern, html.lower())
    if ogrn:
        ogrns.append(ogrn[0])
    else:
        ogrns.append(False)

####Дата собрания

In [ ]:
dates = []
for html in data:
    raw_date = re.findall(r'(?<=проведения ).+?20\d{2}', html.lower())
    if len(raw_date) == 0: # если не указана дата проведения собрания, то возьмем дату наступления события
        date = re.findall(r'(?<=Дата наступления события \(существенного факта\), о котором составлено сообщение \(если применимо\): ).+?20\d{2}', html)
        dates.append(date[0])
    else:
        date = re.findall(r'\d{2}.\d{2}.\d{4}|\d{2} [a-я]+? \d{4}|«\d{2}» [a-я]+? \d{4}|\d{2}[a-я]+? \d{4}|\d{1} [a-я]+? \d{4}', raw_date[0])
        if len(date) == 0:
            dates.append(False)
        else:
            dates.append(date[0])

####Форма собрания

In [ ]:
forms = []
for html in data:
    html = html.lower()
    form = re.findall(r'(?<=форма проведения общего собрания)[0-9а-яА-ЯёЁ\(\)\«\»\.\"\s\–\–\-]+?:([а-яА-Я\s,]+)\.? дата', html)
    if len(form) == 0:
      form = re.findall(r'(?<=форма проведения общего собрания)[0-9а-яА-ЯёЁ\(\)\«\»\.\"\s\–\–\-]+?:([а-яА-Я\s,]+)\.? адрес', html)
      if len(form) == 0:
        form = re.findall(r'(?<=форма проведения общего собрания)[0-9а-яА-ЯёЁ\(\)\«\»\.\"\s\–\–\-]+?[–-]([а-яА-Я\s,]+)\.?', html)
        if len(form) == 0:
          form = re.findall(r'(?<=форма проведения общего собрания)[а-яА-ЯёЁ\(\)\«\»\.\"\s\–\–\-]+?:([а-яА-Я\s,]+)\.?', html)
          if len(form) == 0:
            form = re.findall(r'(?<=форма проведения общего собрания):([а-яА-Я\s,]+) дата', html)
            if len(form) == 0:
              form = re.findall(r'(?<=форма проведения внеочередного общего собрания)[0-9а-яА-ЯёЁ\(\)\«\»\.\"\s\–\–\-]+?[:-]([а-яА-Я\s,]+)\.?', html)
              if len(form) == 0:
                form = re.findall(r'(?<=форма проведения общего собрания):([а-яА-Я\s,]+).+? дата', html)
                if len(form) == 0:
                  form = re.findall(r'(?<=форма проведения общего собрания):([а-яА-Я\s,]+).+?2\.', html)
                  if len(form) == 0:
                    form = re.findall(r'(?<=форма проведения)\s?собрания?:([а-яА-Я\s,]+)\.? дата?', html)
                    if len(form) == 0:
                      form = re.findall(r'(?<=форма проведения общего собрания акционеров эмитента)([а-яА-Я\s,]+)', html)
                      if len(form) == 0:
                        form = re.findall(r'(?<=форма проведения общего собрания)[а-я\(\)\.;\s]+?;?([а-яА-Я\s,]+) дата?', html)
                        if len(form) == 0:
                          form = re.findall(r'(?<=форма проведения общего собрания)\.?([а-яА-Я\s,]+)', html)
                          if len(form) == 0: 
                            form = re.findall(r'(?<=форма проведения внеочередного общего собрания):([а-яА-Я\s,]+)\.?', html)
                            if len(form) == 0:
                              form = re.findall(r'(?<=форма проведения годового общего собрания акционеров):([а-яА-Я\s,]+)\.?', html)
                              if len(form) == 0:
                                form = re.findall(r'(?<=форма проведения):([а-яА-Я\s,]+)\.?\s?дата?', html)
                                if len(form) == 0:
                                  form = re.findall(r'(?<=форма проведения собрания):([а-яА-Я\s,]+)\.? 2?', html)
                                  if len(form) == 0:
                                    form = re.findall(r'(?<=форма проведения собрания акционеров эмитента):([а-яА-Я\s,]+)', html)
                                    if len(form) == 0:
                                      form = re.findall(r'(?<=форма проведения внеочередного общего собрания акционеров )–([а-яА-Я\s,\(\)]+)..2\.', html)
                                      if len(form) == 0:    
                                        form = re.findall(r'(?<=форма проведения собрания)[0-9а-яА-ЯёЁ\(\)\«\»\.\"\s\–\–\-]+?:([а-яА-Я\s,]+)\.', html)
                                        if len(form) == 0:
                                          forms.append(False)
                                          continue
    forms.append(form[0].strip())

####Тип отчетности, которую поручено проверять аудитору

In [ ]:
doc_types = []
for html in data:
    html = html.lower()
    doc_type = re.findall(r'(?<=проверк)[уие] (.{1,40}?отчетности)', html)
    if len(doc_type) == 0:
      doc_type = re.findall(r'(?<=аудит)[ау] (.{1,40} отчетности)', html)
      if len(doc_type) == 0:
        doc_type = re.findall(r'(?<=проверк)[уие] (.{1,40}?деятельности)', html)
        if len(doc_type) == 0:
          doc_type = re.findall(r'(?<=проверк)[уие] (деятельности.{1,10}?) ', html)
          if len(doc_type) == 0:
                                          doc_types.append('Не указано')
                                          i +=1
                                          continue
    doc_types.append(doc_type[0].strip())

####Наименование аудитора

In [ ]:
audit_names = []
i = 0
k = 0
for html in data:
    html = html.lower()
    audit_name = re.findall(r'(?<=утвердить аудитором общества )-([а-яё \"«»-]{1,35})', html)
    if len(audit_name) == 0:
      audit_name = re.findall(r'(?<=утвердить аудитором общества на 202)[10] г\.?\s?([а-яё \"«-]{1,100}\»?)', html)
      if len(audit_name) == 0:
        audit_name = re.findall(r'(?<=утвердить аудитором)[^.:]+?(«[а-яё \"\.«»-]{1,50}»)', html)
        if len(audit_name) == 0:
          audit_name = re.findall(r'(?<=утвердить аудитором общества)[: ][^.:]+?(«[а-яё \"«»-]{1,35}»)', html)
          if len(audit_name) == 0:
            audit_name = re.findall(r'(?<=утвердить аудитора общества)[: ][^.:]+?([«\"][а-яё \"«»-]{1,35}[»\"])', html)
            if len(audit_name) == 0:
              audit_name = re.findall(r'(?<=утвердить аудитором)[^.:]+?(\"[а-яё \"«»\-]{1,35}\b\")', html)
              if len(audit_name) == 0:
                audit_name = re.findall(r'(?<=утвердить в качестве аудитора)[^.:]+?(«[а-яё \"«»-]{1,35}»)', html)
                if len(audit_name) == 0:
                  audit_name = re.findall(r'(?<=утвердить)[^.:]+?(\«[А-Яа-яё \"«»\-]{1,35}\b\») аудитором', html)
                  if len(audit_name) == 0:
                    audit_name = re.findall(r'(?<=утвердить в качестве аудитора)[^:]+?([\"«][а-яё \"«»-]{1,35}[\"»])', html)
                    if len(audit_name) == 0:
                      audit_name = re.findall(r'(?<=утвердить аудитором общества)[: ][^.:]+?([\"«][а-яё0-9 \"«»-]{1,35}[\"»])', html)
                      if len(audit_name) == 0:
                        audit_name = re.findall(r'(?<=утвердить)[^.:]+?([\"«“][А-Яа-яё \"«»\-]{1,35}\b[\"»”])[^.:]+?в качестве аудитора', html)
                        if len(audit_name) == 0:
                          audit_name = re.findall(r'(?<=утвердить аудиторскую фирму)[^.:]+?([\"«][А-Яа-яё \"«»\-]{1,35}\b[\"»])', html)#
                          if len(audit_name) == 0:
                            audit_name = re.findall(r'(?<=одобрить утверждение в качестве аудитора)[^.:]+?([\"«][А-Яа-яё \"«»\-]{1,35}\b[\"»])', html)#
                            if len(audit_name) == 0:
                              audit_name = re.findall(r'(?<=утвердить аудитора)[^.:]+?([\"«][А-Яа-яё \"«»\-]{1,35}\b[\"»])', html)#
                              if len(audit_name) == 0:     
                                          audit_names.append('Не указано')
                                          continue                                     
    audit_names.append(audit_name[0].strip())

####ИНН аудитора

In [ ]:
auditorPattern = re.compile(r'.{300}(?<=утверд).{300}')
aud_innPattern = re.compile(r'(?<=инн: )\d{10}\b|(?<=инн )\d{10}\b|(?<=инн эмитента [-\–] )\d{10}\b')
aud_inns = []
for html in data:
    auditor = re.findall(auditorPattern, html.lower())
    if auditor:
        inn_a = re.findall(aud_innPattern, auditor[0])
        if inn_a:
          aud_inns.append(inn_a[0])
        else:
          aud_inns.append('Не указано')
    else:
          aud_inns.append('Не указано')

####Утвержденный состав совета директоров

In [ ]:
directors = []
for html in data:
    director_raw = re.findall(r'избрать в состав Совета директоров.+|состав Совета директоров.+|Совет директоров.+', html)
    if len(director_raw) == 0:
        directors.append('Не указано')
    else:
        director = re.findall(r'(?<=следующих кандидатов: )(.+?) Результаты голосования', director_raw[0])
        if len(director) == 0:
            director = re.findall(r'(?<=Избрать в Совет директоров Общества: )(.+?) Вопрос', director_raw[0])
            if len(director) == 0:
                director = re.findall(r'(?<=набравших наибольшее число голосов )(.+?) Вопрос', director_raw[0])
                if len(director) == 0:
                    director = re.findall(r'(?<=Совет директоров ПАО «Завод «Ладога»: )(.+?) Дата составления', director_raw[0])
                    if len(director) == 0:
                        director = re.findall(r'(?<=состав Совета директоров Общества: )(.+?)»\. По вопросу', director_raw[0])
                        if len(director) == 0:
                            director = re.findall(r'(?<=в следующем составе: )(.+?)\. Число голосов, которыми обладали лица, включенные в список лиц, имевших право на участие в общем собрании, для голосования по данному вопросу повестки дня', director_raw[0])
                            if len(director) == 0:
                                director = re.findall(r'(?<=в следующем составе: )(.+?) По четвертому вопросу повестки дня:', director_raw[0])
                                if len(director) == 0:
                                    director = re.findall(r'(?<=в следующем составе: )(.+?) По вопросу №2 повестки дня:', director_raw[0])
                                    if len(director) == 0:
                                        director = re.findall(r'(?<=в следующем составе: )(.+?) Результаты голосования:', director_raw[0])
                                        if len(director) == 0:
                                            director = re.findall(r'(?<=следующих кандидатов: )(.+?) Вопрос 3\.', director_raw[0])
                                            if len(director) == 0:
                                                director = re.findall(r'(?<= человек: )(.+?) 2\.7\. ', director_raw[0])
                                                if len(director) == 0:
                                                    director = re.findall(r'(?<=в составе: )(.+?) Решение принято единогласно\.', director_raw[0])
                                                    if len(director) == 0:
                                                        director = re.findall(r'(?<=в следующем составе: )(.+?) ВОПРОС № 3', director_raw[0])
                                                        if len(director) == 0:
                                                            director = re.findall(r'(?<=в следующем составе: )(.+?) По пятому вопросу повестки дня', director_raw[0])
                                                            if len(director) == 0:
                                                                director = re.findall(r'(?<=в следующем составе: )(.+?) Голосовали:', director_raw[0])
                                                                if len(director) == 0:
                                                                    director = re.findall(r'(?<=в следующем составе: )(.+?) 2\.7\.', director_raw[0])
                                                                    if len(director) == 0:
                                                                        director = re.findall(r'(?<=в следующем составе: )(.+?) По вопросу № 3 повестки дня:', director_raw[0])
                                                                        if len(director) == 0:
                                                                            director = re.findall(r'(?<=в следующем составе: )(.+?) Число голосов, которыми по данному вопросу повестки дня обладали все лица, включенные в список', director_raw[0])
                                                                            if len(director) == 0:
                                                                                director = re.findall(r'(?<=состав Совета директоров ПАО «Уралкалий»: )(.+?) ВОПРОС ПОВЕСТКИ ДНЯ № 4:', director_raw[0])
                                                                                if len(director) == 0:
                                                                                    director = re.findall(r'(?<=следующих лиц: )(.+?) 2\.6\.5\.', director_raw[0])
                                                                                    if len(director) == 0:
                                                                                        director = re.findall(r'(?<=членов Совета директоров: )(.+?) 2\.6\.3\.', director_raw[0])
                                                                                        if len(director) == 0:
                                                                                            director = re.findall(r'(?<=следующих лиц: )(.+?) Число голосов, которыми обладали лица, включенные в список лиц, имевших право на участие в общем собрании', director_raw[0])
                                                                                            if len(director) == 0:
                                                                                                director = re.findall(r'(?<=в составе: )(.+?) 2\.7\. Дата составления и номер протокола общего собрания участников \(акционеров\) эмитента Протокол внеочередного общего собрания акционеров №1', director_raw[0])
                                                                                                if len(director) == 0:
                                                                                                    director = re.findall(r'(?<=из следующих лиц: )(.+?) 2\.7\. Дата составления и номер протокола общего собрания акционеров эмитента:', director_raw[0])
                                                                                                    if len(director) == 0:
                                                                                                        director = re.findall(r'(?<=в составе: )(.+?) РЕШЕНИЕ ПРИНЯТО', director_raw[0])
                                                                                                        if len(director) == 0:
                                                                                                            director = re.findall(r'(?<=следующих кандидатов: )(.+?) 2\.7\. Дата составления и номер протокола внеочередного общего собрания:', director_raw[0])
                                                                                                            if len(director) == 0:
                                                                                                                director = re.findall(r'(?<=:)(.+?)Информация не раскрывается', director_raw[0])
                                                                                                                if len(director) == 0:
                                                                                                                    director = re.findall(r'(?<=в следующем составе: )(.+?) 5\. Избрать ревизионную комиссию', director_raw[0])
                                                                                                                    if len(director) == 0:
                                                                                                                        director = re.findall(r'(?<=в количестве \d{1} человек )(.+?) В соответствии с требованиями', director_raw[0])
                                                                                                                        if len(director) == 0:
                                                                                                                            director = re.findall(r'(?<=следующих кандидатов: )(.+?) По шестому вопросу повестки дня:', director_raw[0])
                                                                                                                            if len(director) == 0:
                                                                                                                                director = re.findall(r'(?<=следующих лиц: )(.+?) Фамилия, имя, отчество кандидата', director_raw[0])
                                                                                                                                if len(director) == 0:
                                                                                                                                    director = re.findall(r'(?<=в следующем составе: )(.+?) В голосовании приняло участие', director_raw[0])
                                                                                                                                    if len(director) == 0:
                                                                                                                                        director = re.findall(r'(?<=в следующем составе - )(.+?) Вопрос повестки дня № 4:', director_raw[0])
                                                                                                                                        if len(director) == 0:
                                                                                                                                            director = re.findall(r'(?<=следующих членов: )(.+?) РЕШЕНИЕ ПРИНЯТО', director_raw[0])
                                                                                                                                            if len(director) == 0:
                                                                                                                                                director = re.findall(r'(?<=Совет директоров либо могут быть сданы в Общество)()', director_raw[0])
                                                                                                                                                if len(director) == 0:
                                                                                                                                                    director = re.findall(r'(?<=в составе: )(.+?) 2\.7\. Дата составления и номер протокола общего собрания участников \(акционеров\) эмитента:', director_raw[0])
                                                                                                                                                    if len(director) == 0:
                                                                                                                                                        director = re.findall(r'(?<=в следующем составе: )(.+?)"\. 7\. По СЕДЬМОМУ ВОПРОСУ:', director_raw[0])
                                                                                                                                                        if len(director) == 0:
                                                                                                                                                            directors.append('Не указано')
                                                                                                                                                            continue
                                                        
        directors.append(director[0])

####Выплата дивидендов

In [ ]:
divPattern = re.compile(r'.{200}(?<=дивиденд).{200}')
divs = []
for html in data:
    div = re.findall(divPattern, html.lower())
    if div:
        divs.append(div[0])
    else:
        divs.append(False)
answers = []
for html in data:
    if html.lower().find('дивиденд') > 0:
        if html.lower().find('выплатить дивиденды') > 0 or html.lower().find('осуществить выплату дивидендов') > 0 or html.lower().find('выплату дивидендов осуществить') > 0:
            answers.append('принято решение выплатить дивиденды')
            continue
        elif html.lower().find('не выплачивать') > 0:
            answers.append('принято решение не выплачивать дивиденды')
            continue
    answers.append('вопрос не поднимался')

####Таблица

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
table = pd.DataFrame({'ID': range(1, 304), 'Текст':files, 'Полное наименование': full_names,'Сокращенное наименование': names, 'Адрес': addresses,'ИНН эмитента': inns, 'ОГРН эмитента': ogrns, 'Дата собрания': dates, 'Форма собрания': forms, 'Наименование аудитора':audit_names, 'ИНН аудитора':aud_inns, 'Тип отчетности для проверки аудитору':doc_types, 'Утвержденный состав совета директоров': directors,'Выплата дивидендов': answers})
table

,ID,Текст,Полное наименование,Сокращенное наименование,Адрес,ИНН эмитента,ОГРН эмитента,Дата собрания,Форма собрания,Наименование аудитора,ИНН аудитора,Тип отчетности для проверки аудитору,Утвержденный состав совета директоров,Выплата дивидендов
0,1,"[\n, html, \n, [\n, [\n, <title>Интерфакс – Се...","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СИБН...","ООО ""СИБНЕФТЕХИМТРЕЙД""","625055, Тюменская обл., г. Тюмень, ул. Пражска...",7206033362,1067206072108,13.01.2021,собрание,Не указано,Не указано,Не указано,Не указано,вопрос не поднимался
1,2,"[\n, html, \n, [\n, [\n, <title>Интерфакс – Се...","Акционерное общество ""Коминефтегеофизика""","АО ""Коминефтегеофизика""","Республика Коми, г. Ухта",1102015174,1021100730672,22.01.2021,заочное голосование,Не указано,Не указано,Не указано,Не указано,вопрос не поднимался
2,3,"[\n, html, \n, [\n, [\n, <title>Интерфакс – Се...","Открытое акционерное общество ""Пензенский заво...","ОАО ""Пензкомпрессормаш""","440015 Россия, г. Пенза, ул. Аустрина, 63",5835000698,1025801203537,«30» декабря 2020,заочное голосование,Не указано,Не указано,Не указано,Не указано,вопрос не поднимался
3,4,"[\n, html, \n, [\n, [\n, <title>Интерфакс – Се...","Акционерное Общество ""Канская сортоиспытательн...","АО ""КСС""","Россия, Красноярский край, Канский район, село...",2450021526,1062450011304,16 января 2021,собрание,Не указано,Не указано,Не указано,Не указано,вопрос не поднимался
4,5,"[\n, html, \n, [\n, [\n, <title>Интерфакс – Се...",Общество с ограниченной ответственностью «Маги...,ООО «Магистраль двух столиц»,"Российская Федерация, 196158, Санкт-Петербург,...",7703793405,1137746608383,19 января 2021,совместное присутствие,Не указано,Не указано,Не указано,1. Бигерт Кристиан; 2. Бронвальд Седрик; 3. Ви...,вопрос не поднимался
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,299,"[\n, html, \n, [\n, [\n, <title>Интерфакс – Се...","Акционерное общество ""ОДК-СТАР""","АО ""ОДК-СТАР""","614990, г. Пермь, ул. Куйбышева, д. 140 а",5904100329,1025900895712,22 января 2021,собрание,Не указано,Не указано,Не указано,Не указано,вопрос не поднимался
299,300,"[\n, html, \n, [\n, [\n, <title>Интерфакс – Се...","Закрытое акционерное общество ""РАЗНООПТТОРГ""","ЗАО ""РАЗНООПТТОРГ""","127055, Москва г, Сущёвская ул, дом 12, строен...",7707004690,1027700079736,16 апреля 2020,собрание,«новый консультант»,Не указано,Не указано,Звонова Лидия Григорьевна Галкин Валерий Никол...,вопрос не поднимался
300,301,"[\n, html, \n, [\n, [\n, <title>Интерфакс – Се...",Акционерное общество «Гусевский стекольный зав...,АО «СЗД»,"Российская Федерация, 601508, Владимирская обл...",3304001065,1033300200383,25 июня 2019,собрание,«гусевский стекольный завод имени ф.э. дзержин...,Не указано,Не указано,1.Климашин Александр Васильевич 2.Верховых Над...,принято решение не выплачивать дивиденды
301,302,"[\n, html, \n, [\n, [\n, <title>Интерфакс – Се...",Акционерное общество «Эталон ЛенСпецСМУ»,АО «Эталон ЛенСпецСМУ»,"Российская Федерация, Санкт-Петербург",7802084569,1027801544308,19 марта 2021,собрание,Не указано,7814116230,Не указано,Не указано,вопрос не поднимался


In [ ]:
table.to_excel("project_advb.xlsx")

In [ ]:
table.to_csv("project_advb.csv")